## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...
1,2025-08-18-04-01-11 +0000,nyt,"Trump’s Tariffs Will Crush India’s Exporters, ...",https://www.nytimes.com/2025/08/18/business/tr...
2,2025-08-18-04-01-06 +0000,nyt,Trump Is Pushing India Back Toward China,https://www.nytimes.com/2025/08/18/world/asia/...
3,2025-08-18-04-01-04 +0000,nyt,A New Way to Reduce Children’s Deaths: Cash,https://www.nytimes.com/2025/08/18/health/cash...
4,2025-08-18-03-57-57 +0000,wapo,U.S. ready to give Ukraine security guarantees...,https://www.washingtonpost.com/world/2025/08/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2392/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,26
95,putin,15
32,ukraine,14
1,zelensky,11
4,russia,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
0,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...,83
127,2025-08-17-12-18-22 +0000,nypost,"European leaders, NATO boss will back up Zelen...",https://nypost.com/2025/08/17/world-news/europ...,77
33,2025-08-18-00-15-39 +0000,bbc,Washington talks could prove more vital for Uk...,https://www.bbc.com/news/articles/ckglxlx5vldo...,71
8,2025-08-18-03-26-59 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...,71
162,2025-08-17-06-42-55 +0000,bbc,"Russia complicating end to war, says Zelensky,...",https://www.bbc.com/news/articles/c04rv2p3936o...,69


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
0,83,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...
114,52,2025-08-17-14-39-33 +0000,nypost,Putin agreed to NATO-style US security guarant...,https://nypost.com/2025/08/17/us-news/putin-ag...
87,31,2025-08-17-18-52-45 +0000,nypost,Nationwide strike erupts across Israel demandi...,https://nypost.com/2025/08/17/world-news/natio...
52,30,2025-08-17-21-45-24 +0000,nypost,New Yorkers fighting against massive battery s...,https://nypost.com/2025/08/17/us-news/new-york...
74,30,2025-08-17-20-01-23 +0000,nypost,NYC hookah lounge where 12 were shot Sunday wa...,https://nypost.com/2025/08/17/us-news/nyc-hook...
99,30,2025-08-17-17-18-00 +0000,wsj,More National Guard troops are heading soon to...,https://www.wsj.com/politics/national-security...
71,26,2025-08-17-20-13-47 +0000,nyt,Air Canada Flight Attendants Say They Will Def...,https://www.nytimes.com/2025/08/17/world/canad...
142,25,2025-08-17-10-00-00 +0000,latimes,News Analysis: Newsom's decision to fight fire...,https://www.latimes.com/california/story/2025-...
115,24,2025-08-17-14-37-47 +0000,nypost,HUD launches English-only initiative for all d...,https://nypost.com/2025/08/17/us-news/hud-laun...
53,24,2025-08-17-21-37-05 +0000,nypost,How downtown Los Angeles became a boarded-up g...,https://nypost.com/2025/08/17/us-news/how-down...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
